In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from typing import List
import re

In [19]:
from dotenv import load_dotenv
import os
from openai import OpenAI


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

In [52]:
def extract_html_from_web_page(url: str):
    options = Options()
    options.headless = True  # Run in headless mode (no browser UI)

    # Initialize the WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    try:
        # Navigate to a URL in the domain to set cookies
        driver.get(url)

        # Add cookies from the curl command
        driver.add_cookie({'name': '_wbauid', 'value': '4539604231702381935'})
        driver.add_cookie({'name': 'CrmToken', 'value': '402f57dd0a0627d6c644806308f0ecab966d7cd383c015820432dc17913553c0c6bc45047335765ee6ee1a347faa3b59a75c5b7b5e22f2a259f19588'})

        # Now navigate to the desired page
        # url = 'http://crm-front.alljobswb.svc.k8s.stage-dp/account'
        # driver.get(url)

        # element = WebDriverWait(driver, 10).until(
        #     EC.presence_of_element_located((By.ID, "profile"))
        # )

        # Get the page source
        html_content = driver.page_source
        return html_content

    finally:
        # Close the browser
        driver.quit()
    

In [100]:
detail_page_url_1 = 'https://www.endress.com/en/field-instruments-overview/level-measurement/Float-switch-Liquifloat-FTS20?t.tabId=product-overview'
detail_page_url_2 = 'https://www.galco.com/cd1-k-400-30-131427.html'
detail_page_url_3 = 'https://www.nriparts.com/products/masterflex-7017-20-miscellaneous/268086'
detail_page_url_4 = 'https://www.palmindustrial.com/products/cole-parmer-masterflex-7017-20-peristaltic-pump-head'
detail_page_url_5 = 'https://www.walkerindustrial.com/Bircher-Reglomat-212174-p/212174.htm'

detail_page_url = detail_page_url_5


detail_page_content = extract_html_from_web_page(detail_page_url)
detail_page_content[:4000]

'<html class="no-js ie9 productdetails" lang="en"><!--<![endif]--><head><meta http-equiv="origin-trial" content="Az520Inasey3TAyqLyojQa8MnmCALSEU29yQFW8dePZ7xQTvSt73pHazLFTK5f7SyLUJSo2uKLesEtEa9aUYcgMAAACPeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZyIsImV4cGlyeSI6MTcyNTQwNzk5OSwiaXNTdWJkb21haW4iOnRydWUsImlzVGhpcmRQYXJ0eSI6dHJ1ZX0=">\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\n<script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=G-KT7DRTSFL5&amp;l=dataLayer&amp;cx=c"></script><script src="//bat.bing.com/bat.js" async=""></script><script type="text/javascript" id="zsiqscript" defer="" src="https://salesiq.zoho.com/widget"></script><script src="//bat.bing.com/bat.js" async=""></script><script type="text/javascript" async="" src="https://www.gstatic.com/r

In [84]:
def pre_cleaning(soup):
    for script in soup(["script", "footer", "nav", "head", "style"]):  # remove all javascript code
        script.decompose()

In [101]:
soup = BeautifulSoup(detail_page_content, 'html.parser')

pre_cleaning(soup)

In [86]:

def search_html_with_parents(soup, search_terms: List[str], max_token_length: int, parent_depth: int = 1) -> List[str]:
    """
    Search for the provided terms in the given HTML content and return a list of relevant elements, including parent elements.

    :param html_content: A string containing the HTML content.
    :param search_terms: A list of search terms, ranked in order of estimated relevance.
    :param max_token_length: The maximum token length for the final list.
    :param parent_depth: The depth of parent elements to include.
    :return: A list of HTML elements that contain the search terms, along with their parent elements.
    """
    # Dictionary to hold search results: {term: [elements]}
    search_results = {term: [] for term in search_terms}

    for term in search_terms:
        # Find all elements containing the term
        for element in soup.find_all(text=re.compile(re.escape(term), re.IGNORECASE)):
            current_element = element.parent
            # Traverse up to the specified parent depth
            for _ in range(parent_depth):
                if current_element.parent is not None:
                    current_element = current_element.parent
            search_results[term].append(str(current_element))

    # Populate the final list with elements, prioritizing earlier terms
    final_list = []
    total_tokens = 0

    for term in search_terms:
        for element in search_results[term]:
            element_tokens = len(element.split())
            if total_tokens + element_tokens > max_token_length:
                return final_list  # Return the list if adding the element would exceed the token limit
            final_list.append(element)
            total_tokens += element_tokens

    return final_list

In [87]:

# Example usage with parent depth:
popular_currency_symbols = ["$", "€", "£", "¥", "₹", "₽", "₩", "₣", "C$", "A$", "R$", "₺", "RM", "฿", "₱", "S$", "HK$", "NZ$", "kr", "zł"]
popular_currency_codes = ["USD", "EUR", "GBP", "JPY", "INR", "RUB", "KRW", "CHF", "CAD", "AUD", "BRL", "TRY", "MYR", "THB", "PHP", "SGD", "HKD", "NZD", "SEK", "NOK", "DKK", "PLN"]


search_terms = popular_currency_symbols + popular_currency_codes
max_token_length = 50  # Example token length limit
parent_depth = 1

matched_elements = search_html_with_parents(soup, search_terms, max_token_length, parent_depth)
matched_elements = list(set(matched_elements))
matched_elements

/tmp/ipykernel_1398597/511472134.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=re.compile(re.escape(term), re.IGNORECASE)):


['<div class="col-4">\n\t\t\t\t\t\t\tMFG #: <strong>${ product.model }</strong>\n</div>',
 '<div class="col-4">\n\t\t\t\t\t\t\tProduct ID: <strong>${ getProductId() }</strong>\n</div>']

In [79]:
matched_elements_str = '\n'.join([f'{idx}: \'{m}\'' for idx, m in enumerate(matched_elements)])
print(matched_elements_str)

0: '<span class="price-wrapper" data-price-amount="1754.5000" data-price-type="finalPrice" id="product-price-7364554">
<span class="price">$1,754.50</span>
</span>'
1: '<div class="modal-content" data-role="content" id="modal-content-29"><div class="modal-component" data-bind="css: modalClass, hasFocus: focused">
<!-- ko if: state() || $data.modal --><!-- /ko -->
</div></div>'


In [80]:
system_message = """
You are automated web-crawler working as part of a product that helps blind people use websites. You have been
provided with a numbered list og HTML elements. Given a directive, your job is to identify the single element that
is most relevant to the directive. Return the number of the element, wrapped in curved parentheses.

[Example 1]
We are given the following elements:
{
  1: '<th scope="row" class="infobox-label"><div style=";">&nbsp;<a href="/wiki/President_of_the_United_States"
  title="President of the United States"></a> </div></th>',
  2: '<th scope="row" class="infobox-label"><div style=";">&nbsp;<a href="/wiki/Vice_President_of_the_United_States"
  title="Vice President of the United States">Vice President</a> </div></th>'
  3: '<tr><th scope="row" class="infobox-label"><a href="/wiki/Left-_and_right-hand_traffic" title="Left- and right-hand traffic">
  Driving side</a></th><td class="infobox-data">right<sup id="cite_ref-drive_23_0" class="reference"><a 
  href="#cite_note_drive-23">[h]</a></sup></td></tr>'
}
And the following directive
"Find an element that relates to the driving side in the United States"

In this case, we can see that the third element contains the information we`re looking for, so we should return:
'(3)'

Keep in mind that the innerText of an element is not the only way in which it can relate to a directive. Sometimes the most relevant
element will be a link to a new page whose title seems relevant.

You must always return a number. If you don't find an element that is directly relevant, think abstractly, and consider which
element may be directionally similar to the directive.

For example, let's take Example 1 again, but with a new directive:
"Find information about the population of Washington D.C."

In this case, none of the elements are directly relevant, but the first element is directionally similar, because the President of
the United States lives in Washington D.C. So we should return:
'(1)'
"""

user_request = """
As an automated web-crawler, please find relevant price and manufacturer details from the following elements:
{{
  {matched_elements}
}}
""".format(matched_elements=matched_elements_str)

In [81]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "assistant", "content": system_message},
        {"role": "user", "content": user_request}
    ],
    stream=False,
)
# response

In [82]:
print(response.choices[0].message.content)

The directive to find price and manufacturer details is most relevant to the first element, thus return:
'(0)'


In [50]:
print(user_request)


As an automated web-crawler, please find relevant price and manufacturer details from the following elements:
{
  0: '<div class="eh-price-widget--price">
<span class="eh-price-widget--price-prefix eh-label eh-font-metrics--14-22">from</span>
<span class="eh-price-widget--price-formatted eh-label eh-font-metrics--20-26">€59.00</span>
</div>'
1: '<div class="eh-price-widget--price">
<span class="eh-price-widget--price-prefix eh-label eh-font-metrics--14-22">from</span>
<span class="eh-price-widget--price-formatted eh-label eh-font-metrics--20-26">€67.00</span>
</div>'
2: '<div class="eh-quickselector--product-price-scale-price-value eh-font-bold">
<span>€67.00</span>
</div>'
3: '<div class="eh-quickselector--product-price-scale-price-value eh-font-bold">
<span>€63.00</span>
</div>'
4: '<div class="eh-quickselector--product-price-scale-price-value eh-font-bold">
<span>€59.00</span>
</div>'
}



In [98]:
print(soup.get_text().replace('  ', '').replace('\n\n', ''))


NEED HELP? CALL US AT 949.446.6757Log inorCreate account
Cart
0US DollarEuroBritish Pound SterlingCanadian DollarAustralian Dollar  USD  US Dollar  Euro  British Pound Sterling  Canadian Dollar  Australian Dollar
Search
MenuUS DollarEuroBritish Pound SterlingCanadian DollarAustralian Dollar  USD  US Dollar  Euro  British Pound Sterling  Canadian Dollar  Australian DollarCart 0FIND YOUR PART
+-
PLC's
AUTOMATION
COMPUTER PARTS
CONNECTORS
ELECTRICAL
CIRCUIT BREAKERS
SENSORS
SWAGELOK
FITTINGS
VALVES
LAB EQUIPMENT
TEST EQUIPMENT
PNEUMATICS
POWER SUPPLIES
PUMPS
BEARINGS
NETWORKINGABOUT PALM INDUSTRIAL
PRODUCT GUARANTEE
Sell Your Surplus
OUR PARENT COMPANYLog in
Create account
SearchNEED HELP? CALL US AT 949.446.6757
FIND YOUR PART
PLC's
AUTOMATION
COMPUTER PARTS
CONNECTORS
ELECTRICAL
CIRCUIT BREAKERS
SENSORS
SWAGELOK
FITTINGS
VALVES
LAB EQUIPMENT
TEST EQUIPMENT
PNEUMATICS
POWER SUPPLIES
PUMPS
BEARINGS
NETWORKING
ABOUT PALM INDUSTRIAL
PRODUCT GUARANTEE
Sell Your Surplus
OUR PARENT COMPANYcol

In [102]:
# Regular expression pattern to match URLs ending in .pdf
pdf_regex = re.compile('.+\.pdf$')

# Find all links that match the regex pattern
pdf_links = [link.get('href') for link in soup.find_all('a', href=pdf_regex)]

print(pdf_links)

['/v/vspfiles/pdf/datasheet/Bircher/SpotScan-Datasheet.pdf', '/v/vspfiles/pdf/datasheet/Bircher/SpotScan-Usermanual.pdf']


In [103]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "assistant", "content": system_message},
        {"role": "user", "content": user_request}
    ],
    stream=False,
)
# response

In [104]:
response

ChatCompletion(id='chatcmpl-8ZvhLbXfH0Un0BLrmyevGAIUIzeKo', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="'(0)'", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1703573963, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=632, total_tokens=635))